CRM数据的可视化分析

检查后发现已完成的两条数据是由于ERP系统登记错误导致，而其余八条数据未完成发货，数据没有问题，后期联系会计人员在ERP系统中进行数据修正即可。CRM中的数据是正确的。接下来，对CRM数据进行可视化分析，以了解大致的销售情况。

1. 分析不同运输方式的销售收入(即订单金额)，判断不同运输方式在销售上的有效性，结合时间信息进一步分析了解不同运输方式的销售趋势；

2. 分析发货地址所在城市不同运输方式的销售收入，了解在不同城市使用不同运输方式的有效性；

3. 分析不同发货地址所在城市的销售收入，了解在不同城市的销售情况，以便优化销售策略

In [ ]:
# 分析不同运输方式的销售收入
import pandas as pd

CRM_df = pd.read_excel(r"C:\Users\lizhi\Desktop\西财\大数据与会计\2023秋\Week 6\CRM订单数据.xlsx") 


import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] #解决中文显示问题
plt.rcParams['axes.unicode_minus']=False #解决不能显示负号的问题

"""
使用pandas自带的DataFrame.plot功能画图，对于pandas DataFrame可视化而言更简便，它根据数据的每一列绘制指定图形，
大部分涉及参数的功能与plt绘图中的函数或参数相似

DataFrame.plot详细参考资料可见：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

"""
sales_by_trans = CRM_df.groupby(['运输方式'])['订单金额'].sum()

sales_by_trans.plot(kind='pie', figsize=(6,6),autopct='%.2f%%',title='不同运输方式的销售额对比',
                 explode=(0,0,0.2),subplots=True,pctdistance=0.8,radius=0.9,labeldistance=1.1,shadow=True,ylabel='')

In [ ]:
# 进一步分析了解不同运输方式的销售趋势
sales_df1 = CRM_df.set_index("下单时间")
sales_by_trans_m = sales_df1.groupby('运输方式')['开票金额'].resample("M").sum()

print(sales_by_trans_m)

sales_by_trans_m = sales_by_trans_m.unstack(0) #把数据展开为可直接绘图的形式（每个运输方式各成一列）
sales_by_trans_m.fillna(0,inplace=True) #将缺失值填补为0
sales_by_trans_m.plot(kind='line',title='不同运输方式的月度销售额变化折线图')

In [ ]:
# 分析不同运输方式和发货地址所在城市下的销售收入 - 热力图
import seaborn as sns
sales_by_trans_city = CRM_df.groupby(['发货地址所在城市','运输方式'])['开票金额'].sum().unstack(0)
plt.figure(figsize=(10,5))
plt.title('不同城市不同运输方式的销售总额统计热力图')
sns.heatmap(sales_by_trans_city,cmap="Reds")

In [ ]:
# 分析不同城市的销售情况 - 地图可视化
from pyecharts.charts import Map
from pyecharts import options as opts

sales_by_city = CRM_df.groupby('发货地址所在城市')['开票金额'].sum()
data_pair = [(i,sales_by_city[i]/10000) for i in sales_by_city.index] #准备绘图数据，并将数据改为万元为单位

city_sales = (
        Map()
        .add(
                "",
                data_pair,
                "china-cities",
                label_opts=opts.LabelOpts(is_show=False),
                is_map_symbol_show=False
                )
        .set_global_opts(
                title_opts=opts.TitleOpts(title="各城市销售额(万)"),
                visualmap_opts=opts.VisualMapOpts(
                        type_="color",
                         pieces=[{"min": 20,"max": 24},
                                {"min": 25, "max": 29},
                                {"min": 30, "max": 34},
                                {"min": 35, "max": 39}],
                        is_piecewise=True
                        ),
                )
        .render("不同城市销售收入可视化1.html")
        )